In [1]:
import pandas as pd
from itertools import product
from typing import Iterable

pd.options.display.max_columns = 100

saf_test_vectors = dict()
"""Stores test vectors for SaF testing for a part"""
additional_IDDQ: dict[str, list[int]] = dict()
"""Stores number of faults covered by IDDQ testing for a part. Structure: [all,SaF]"""


def generate_test_vectors(n_inputs: int = 0, labels: list[str] = []):
    # labels[0] is MSB label
    if n_inputs:
        return pd.DataFrame(
            [*product(*((False, True) for _ in range(n_inputs)))],
            columns=[chr(65 + _) for _ in reversed(range(n_inputs))],
        )
    return pd.DataFrame(
        [*product(*((False, True) for _ in range(len(labels))))],
        columns=labels,
    )


def validate_checks(df: pd.DataFrame, checks: dict[str, pd.Series], out_col: str = "Q"):
    for var, check in checks.items():
        if out_col not in df.columns:
            raise ValueError(f"Output column {out_col} not in DataFrame")
        tmp = df[var].where(check)
        xored = df[out_col][tmp.notna()] ^ df[var][tmp.notna()]
        if xored.any() and not xored.all():
            raise ValueError(f"Mismatch in {var} check: {xored}")


def calc_cases(df: pd.DataFrame, checks: dict[str, pd.Series]):
    out = pd.DataFrame(index=df.index)
    for var, check in checks.items():
        tmp = df[var].where(check)
        out[f"{var}/gnd"] = tmp.where(tmp == True)
        out[f"{var}/vdd"] = tmp.where(tmp == False)
    out[out.notna()] = 1
    return out.fillna(0).T


def validate_SaF_vectors(saf_cases: pd.DataFrame, test_vectors: Iterable[int]):
    if not saf_cases.iloc[:, test_vectors].any(axis=1).all():
        raise ValueError(
            f"Test vectors {test_vectors} do not cover all checks")
    return test_vectors


# to select test cases:
# - select column with most ones and add it to test vector,
# - repeat, but skip rows, where testing is covered
# - if testing vestor is full, stop

In [2]:
# C17 circuit

df = generate_test_vectors(5)

# define/calculate intermediate signals in the structure
df["n1"] = ~(df.A & df.C)
df["n2"] = ~(df.D & df.C)
df["n3"] = ~(df.n2 & df.B)
df["n4"] = ~(df.n2 & df.E)

# define/calculate outputs
df["Y1"] = ~(df.n1 & df.n3)
df["Y2"] = ~(df.n3 & df.n4)

# define/calculate passthrough masks for signals
checks = {
    "A": (df.C & df.n3),
    "B": (df.n2 & (df.n1 | df.n4)),
    "C": (df.A & df.n3)
    | (df.D & df.B & (df.n1 | df.n4 | df.E))
    | (df.D & df.E & df.n3),
    "D": (df.C & df.B & (df.n1 | df.n4 | df.E)) | (df.C & df.E & df.n3),
    "E": (df.n2 & df.n3),
    "n1": df.n3,
    "n2": (df.B & (df.n1 | df.n4)) | (df.E & df.n3) | (df.B & df.E),
    "n3": df.n1 | df.n4,
    "n4": df.n3,
    "Y1": df.index >= 0,
    "Y2": df.index >= 0,
}

# calculate test vectors coverage for a SaF model
out_c17 = calc_cases(df, checks)
saf_test_vectors["C17"] = validate_SaF_vectors(out_c17, [4, 6, 25, 31])
# A/C, A/E, B/C, C/D, n1/n2 shorts not detected by SaF
# with vectors [4, 6, 25, 31] 3 shorts will increase IDDQ
additional_IDDQ["C17"] = [0, 3]
out_c17

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
A/gnd,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1
A/vdd,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0
B/gnd,0,0,1,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,0,0,0
B/vdd,1,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0
C/gnd,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,1
C/vdd,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,0,0,0,0
D/gnd,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1
D/vdd,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0
E/gnd,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0
E/vdd,1,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
df.iloc[saf_test_vectors["C17"]]

,E,D,C,B,A,n1,n2,n3,n4,Y1,Y2
4,False,False,True,False,False,True,True,True,True,False,False
6,False,False,True,True,False,True,True,False,True,True,True
25,True,True,False,False,True,True,True,True,False,False,True
31,True,True,True,True,True,False,False,True,True,True,False


In [4]:
# NO42 circuit

df = generate_test_vectors(4)

df["n3"] = ~(df.A | df.B)
df["n1"] = ~(df.D | df.C)
df["n2"] = ~(df.n1 & df.n3)

df["Q"] = ~(df.n2)

checks = {
    "A": (~df.B & df.n1),
    "B": (~df.A & df.n1),
    "C": (~df.D & df.n3),
    "D": (~df.C & df.n3),
    "n1": df.n3,
    "n2": df.index >= 0,
    "n3": df.n1,
    "Q": df.index >= 0,
}

validate_checks(df, checks)

out_no42 = calc_cases(df, checks)
saf_test_vectors["NO42"] = validate_SaF_vectors(out_no42, [0, 1, 2, 4, 8])
# n1/n3 short not detected by SaF and all vectors
# vector 1 will increase IDDQ
additional_IDDQ["NO42"] = [1, 1]
out_no42

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
A/gnd,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
A/vdd,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
B/gnd,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
B/vdd,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C/gnd,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
C/vdd,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
D/gnd,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
D/vdd,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
n1/gnd,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
n1/vdd,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0


In [5]:
# NO2 circuit

df = generate_test_vectors(2)

df["Q"] = ~(df.A | df.B)

checks = {
    "A": ~df.B,
    "B": ~df.A,
    "Q": df.index >= 0,
}

validate_checks(df, checks)

out = calc_cases(df, checks)
saf_test_vectors["NO2"] = validate_SaF_vectors(out, [0, 1, 2])
out

,0,1,2,3
A/gnd,0,1,0,0
A/vdd,1,0,0,0
B/gnd,0,0,1,0
B/vdd,1,0,0,0
Q/gnd,1,0,0,0
Q/vdd,0,1,1,1


In [6]:
# NA2 circuit

df = generate_test_vectors(2)

df["Q"] = ~(df.A & df.B)

checks = {
    "A": df.B,
    "B": df.A,
    "Q": df.index >= 0,
}

validate_checks(df, checks)

out = calc_cases(df, checks)
saf_test_vectors["NA2"] = validate_SaF_vectors(out, [1, 2, 3])
# A/B shorts not detected by SaF and all vectors
additional_IDDQ["NA2"] = [1,1]
out

,0,1,2,3
A/gnd,0,0,0,1
A/vdd,0,0,1,0
B/gnd,0,0,0,1
B/vdd,0,1,0,0
Q/gnd,1,1,1,0
Q/vdd,0,0,0,1


In [7]:
# NO3 circuit

df = generate_test_vectors(3)

df["Q"] = ~(df.A | df.B | df.C)

checks = {
    "A": ~(df.B | df.C),
    "B": ~(df.A | df.C),
    "C": ~(df.A | df.B),
    "Q": df.index >= 0,
}

validate_checks(df, checks)

out = calc_cases(df, checks)
saf_test_vectors["NO3"] = validate_SaF_vectors(out, [0, 1, 2, 4])
# A/B short not detected by SaF and all vectors
out

,0,1,2,3,4,5,6,7
A/gnd,0,1,0,0,0,0,0,0
A/vdd,1,0,0,0,0,0,0,0
B/gnd,0,0,1,0,0,0,0,0
B/vdd,1,0,0,0,0,0,0,0
C/gnd,0,0,0,0,1,0,0,0
C/vdd,1,0,0,0,0,0,0,0
Q/gnd,1,0,0,0,0,0,0,0
Q/vdd,0,1,1,1,1,1,1,1


In [8]:
# NA3 circuit

df = generate_test_vectors(3)

df["Q"] = ~(df.A & df.B & df.C)

checks = {
    "A": df.B & df.C,
    "B": df.A & df.C,
    "C": df.A & df.B,
    "Q": df.index >= 0,
}

validate_checks(df, checks)

out = calc_cases(df, checks)
saf_test_vectors["NA3"] = validate_SaF_vectors(out, [3, 5, 6, 7])
# A/B, A/C, B/C shorts not detected by SaF and all vectors
additional_IDDQ["NA3"] = [3,3]
out

,0,1,2,3,4,5,6,7
A/gnd,0,0,0,0,0,0,0,1
A/vdd,0,0,0,0,0,0,1,0
B/gnd,0,0,0,0,0,0,0,1
B/vdd,0,0,0,0,0,1,0,0
C/gnd,0,0,0,0,0,0,0,1
C/vdd,0,0,0,1,0,0,0,0
Q/gnd,1,1,1,1,1,1,1,0
Q/vdd,0,0,0,0,0,0,0,1


In [9]:
df.iloc[saf_test_vectors["NA3"]]

,C,B,A,Q
3,False,True,True,True
5,True,False,True,True
6,True,True,False,True
7,True,True,True,False


In [10]:
docs_faults_files = {
    "C17": "C17.csv",
    "NO42": "NO42.csv",
    "NO2": "NO2.csv",
    "NO3": "NO3.csv",
    "NA2": "NA2.csv",
    "NA3": "NA3.csv"
}

coverages = pd.DataFrame(
    index=["all", "all+IDDQ", "SaF", "SaF+IDDQ"],
    columns=docs_faults_files.keys(),
    dtype=float,
)
pd.options.display.float_format = "{:.2%}".format
for part, f_path in docs_faults_files.items():
    faults = pd.read_csv(f_path, sep=";", index_col=0)

    # coverage vectors:
    def calc_coverage(coverage_vector):
        cov: pd.Series = coverage_vector.any(axis=1)
        return cov[~cov], cov.mean()

    _, coverages.at["all", part] = calc_coverage(faults == 1)
    _, coverages.at["SaF", part] = calc_coverage(
        faults.iloc[:, saf_test_vectors[part]] == 1
    )
    add_all, add_saf = additional_IDDQ.get(part, [0, 0])
    coverages.at["all+IDDQ", part] = coverages.at["all", part] + \
        add_all/faults.shape[0]
    coverages.at["SaF+IDDQ", part] = coverages.at["SaF", part] + \
        add_saf/faults.shape[0]
coverages

,C17,NO42,NO2,NO3,NA2,NA3
all,81.40%,97.73%,100.00%,100.00%,88.89%,78.57%
all+IDDQ,81.40%,100.00%,100.00%,100.00%,100.00%,100.00%
SaF,77.33%,97.73%,100.00%,100.00%,88.89%,78.57%
SaF+IDDQ,79.07%,100.00%,100.00%,100.00%,100.00%,100.00%


In [11]:
# print("Stopień pokrycia wszystkich możliwych błędów przez wektory testowe:")
# print()
# print("C17")
# print(f"- wyznaczone dla modelu SaF: {c17_saf_coverage:.2f}%")
# print(f"- obejmujące wszystkie kombinacje stanów portów wejściowych: {c17_all_coverage:.2f}%")
# print()
# print("NO42")
# print(f"- wyznaczone dla modelu SaF: {no42_saf_coverage:.2f}%")
# print(f"- obejmujące wszystkie kombinacje stanów portów wejściowych: {no42_all_coverage:.2f}%")